<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 3 - 3 Hours </h1>
<h1 style="text-align:center">Long Short Term Memory (LSTM) for Language Modeling</h1>

<b> Student 1:</b> Hanna Johansson 
<b> Student 2:</b> Matteo Fiore
 
 
In this Lab Session,  you will build and train a Recurrent Neural Network, based on Long Short-Term Memory (LSTM) units for next word prediction task. 

Answers and experiments should be made by groups of one or two students. Each group should fill and run appropriate notebook cells. 
Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an pdf document using print as PDF (Ctrl+P). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed by June 9th 2017.

Send you pdf file to benoit.huet@eurecom.fr and olfa.ben-ahmed@eurecom.fr using **[DeepLearning_lab3]** as Subject of your email.

#  Introduction

You will train a LSTM to predict the next word using a sample short story. The LSTM will learn to predict the next item of a sentence from the 3 previous items (given as input). Ponctuation marks are considered as dictionnary items so they can be predicted too. Figure 1 shows the LSTM and the process of next word prediction. 

<img src="lstm.png" height="370" width="370"> 


Each word (and punctuation) from text sentences is encoded by a unique integer. The integer value corresponds to the index of the corresponding word (or punctuation mark) in the dictionnary. The network output is a one-hot-vector indicating the index of the predicted word in the reversed dictionnary (Section 1.2). For example if the prediction is 86, the predicted word will be "company". 



You will use a sample short story from Aesop’s Fables (http://www.taleswithmorals.com/) to train your model. 


<font size="3" face="verdana" > <i> "There was once a young Shepherd Boy who tended his sheep at the foot of a mountain near a dark forest.

It was rather lonely for him all day, so he thought upon a plan by which he could get a little company and some excitement.
He rushed down towards the village calling out "Wolf, Wolf," and the villagers came out to meet him, and some of them stopped with him for a considerable time.
This pleased the boy so much that a few days afterwards he tried the same trick, and again the villagers came to his help.
But shortly after this a Wolf actually did come out from the forest, and began to worry the sheep, and the boy of course cried out "Wolf, Wolf," still louder than before.
But this time the villagers, who had been fooled twice before, thought the boy was again deceiving them, and nobody stirred to come to his help.
So the Wolf made a good meal off the boy's flock, and when the boy complained, the wise man of the village said:
"A liar will not be believed, even when he speaks the truth."  "</i> </font>.    







Start by loading the necessary libraries and resetting the default computational graph. For more details about the rnn packages, we suggest you to take a look at https://www.tensorflow.org/api_guides/python/contrib.rnn

In [1]:
import numpy as np
import collections # used to build the dictionary
import random
import time
import pickle # may be used to save your model 
import matplotlib.pyplot as plt
#Import Tensorflow and rnn
import tensorflow as tf
from tensorflow.contrib import rnn  

# Target log path
logs_path = 'lstm_words'

# Next-word prediction task

## Part 1: Data  preparation

### 1.1. Loading data

Load and split the text of our story

In [2]:
def load_data(filename):
    with open(filename) as f:
        data = f.readlines()
    data = [x.strip().lower() for x in data]
    data = [data[i].split() for i in range(len(data))]
    data = np.array(data)
    data = np.reshape(data, [-1, ])
    print(data)
    return data

#Run the cell 
train_file ='data/story.txt'
train_data = load_data(train_file)
print("Loaded training data...")
print(len(train_data))

['there' 'was' 'once' 'a' 'young' 'shepherd' 'boy' 'who' 'tended' 'his'
 'sheep' 'at' 'the' 'foot' 'of' 'a' 'mountain' 'near' 'a' 'dark' 'forest'
 '.' 'it' 'was' 'rather' 'lonely' 'for' 'him' 'all' 'day' ',' 'so' 'he'
 'thought' 'upon' 'a' 'plan' 'by' 'which' 'he' 'could' 'get' 'a' 'little'
 'company' 'and' 'some' 'excitement' '.' 'he' 'rushed' 'down' 'towards'
 'the' 'village' 'calling' 'out' 'wolf' ',' 'wolf' ',' 'and' 'the'
 'villagers' 'came' 'out' 'to' 'meet' 'him' ',' 'and' 'some' 'of' 'them'
 'stopped' 'with' 'him' 'for' 'a' 'considerable' 'time' '.' 'this'
 'pleased' 'the' 'boy' 'so' 'much' 'that' 'a' 'few' 'days' 'afterwards'
 'he' 'tried' 'the' 'same' 'trick' ',' 'and' 'again' 'the' 'villagers'
 'came' 'to' 'his' 'help' '.' 'but' 'shortly' 'after' 'this' 'a' 'wolf'
 'actually' 'did' 'come' 'out' 'from' 'the' 'forest' ',' 'and' 'began' 'to'
 'worry' 'the' 'sheep,' 'and' 'the' 'boy' 'of' 'course' 'cried' 'out'
 'wolf' ',' 'wolf' ',' 'still' 'louder' 'than' 'before' '.' 'but' 't

### 1.2.Symbols encoding

The LSTM input's can only be numbers. A way to convert words (symbols or any items) to numbers is to assign a unique integer to each word. This process is often based on frequency of occurrence for efficient coding purpose.

Here, we define a function to build an indexed word dictionary (word->number). The "build_vocabulary" function builds both:

- Dictionary : used for encoding words to numbers for the LSTM inputs 
- Reverted dictionnary : used for decoding the outputs of the LSTM into words (and punctuation).

For example, in the story above, we have **113** individual words. The "build_vocabulary" function builds a dictionary with the following entries ['the': 0], [',': 1], ['company': 85],...


In [3]:
def build_vocabulary(words):
    count = collections.Counter(words).most_common()
    dic= dict()
    for word, _ in count:
        dic[word] = len(dic)
    reverse_dic= dict(zip(dic.values(), dic.keys()))
    return dic, reverse_dic


Run the cell below to display the vocabulary

In [4]:
dictionary, reverse_dictionary = build_vocabulary(train_data)
vocabulary_size= len(dictionary) 
print("Dictionary size (Vocabulary size) = ", vocabulary_size)
print("\n")
print("Dictionary : \n")
print(dictionary)
print("\n")
print("Reversed Dictionary : \n" )
print(reverse_dictionary)

Dictionary size (Vocabulary size) =  113


Dictionary : 

{'days': 32, 'thought': 17, 'shepherd': 33, 'before': 18, 'who': 19, 'began': 35, 'when': 20, 'upon': 36, ',': 1, 'he': 6, 'will': 38, 'come': 21, 'that': 67, 'few': 40, 'little': 100, 'pleased': 41, 'not': 42, 'with': 43, 'again': 23, 'some': 24, 'good': 37, 'fooled': 45, 'flock': 46, 'truth': 47, 'but': 25, 'get': 48, 'came': 26, 'it': 50, 'did': 51, 'said': 52, 'lonely': 105, 'time': 27, 'villagers': 11, 'them': 28, 'speaks': 54, 'him': 12, 'stopped': 55, 'village': 29, 'towards': 56, 'a': 2, 'foot': 57, 'afterwards': 58, 'so': 13, 'meet': 59, 'young': 73, ':': 53, 'after': 61, 'sheep,': 62, 'there': 64, 'rather': 63, 'dark': 65, 'course': 66, 'by': 68, 'could': 39, 'plan': 69, 'of': 9, 'meal': 49, 'rushed': 70, 'his': 15, 'boy': 7, 'shortly': 71, 'than': 72, 'still': 44, 'mountain': 74, 'near': 75, 'made': 34, 'tried': 77, 'wise': 78, 'same': 79, 'the': 0, 'at': 80, 'was': 14, 'been': 81, 'to': 8, 'sheep': 82, 'day': 96, 'cr

## Part 2 : LSTM Model in TensorFlow

Since you have defined how the data will be modeled, you are now to develop an LSTM model to predict the word of following a sequence of 3 words. 

### 2.1. Model definition

Define a 2-layers LSTM model.  

For this use the following classes from the tensorflow.contrib library:

- rnn.BasicLSTMCell(number of hidden units) 
- rnn.static_rnn(rnn_cell, data, dtype=tf.float32)
- rnn.MultiRNNCell(,)


You may need some tensorflow functions (https://www.tensorflow.org/api_docs/python/tf/) :
- tf.split
- tf.reshape 
- ...




In [28]:
def lstm_model(x, w, b, n_hidden, n_input):
    
    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])
    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)
    
    # 1-layer LSTM with n_hidden units.
    rnn_cell = rnn.BasicLSTMCell(n_hidden)
    
    # 2-layer LSTM with n_hidden units.
    rnn_cell2 = rnn.BasicLSTMCell(n_hidden)

    # multi-rnn from the two basic lstm cells
    multi_rnn = rnn.MultiRNNCell([rnn_cell, rnn_cell2])

    # generate prediction
    outputs, states = rnn.static_rnn(multi_rnn, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], w['out']) + b['out']
    


Training Parameters and constants

In [29]:
# Training Parameters
learning_rate = 0.001
epochs = 50000
display_step = 1000
n_input = 3

#For each LSTM cell that you initialise, supply a value for the hidden dimension, number of units in LSTM cell
n_hidden = 64

tf.reset_default_graph()

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1], name='InputData')
y = tf.placeholder("float", [None, vocabulary_size], name='Labels')

# LSTM  weights and biases
weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocabulary_size])) }

#build the model
with tf.name_scope('Model'):
    pred = lstm_model(x, weights, biases, n_hidden, n_input)

Define the Loss/Cost and optimizer

In [30]:
with tf.name_scope('Loss'):
    # Loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
with tf.name_scope('RMSPOpt'):    
    #use RMSProp Optimizer
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

We give you here the Test Function

In [31]:
#run the cell
def test(sentence, session, verbose=False):
    sentence = sentence.strip()
    words = sentence.split(' ')
    if len(words) != n_input:
        print("sentence length should be equal to", n_input, "!")
    try:
        symbols_inputs = [dictionary[str(words[i - n_input])] for i in range(n_input)]
        keys = np.reshape(np.array(symbols_inputs), [-1, n_input, 1])
        onehot_pred = session.run(pred, feed_dict={x: keys})
        onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
        words.append(reverse_dictionary[onehot_pred_index])
        sentence = " ".join(words)
        if verbose:
            print(sentence)
        return reverse_dictionary[onehot_pred_index]
    except:
        print(" ".join(["Word", words[i - n_input], "not in dictionary"]))

## Part 3 : LSTM Training  

In the Training process, at each epoch, 3 words are taken from the training data, encoded to integer to form the input vector. The training labels are one-hot vector encoding the word that comes after the 3 inputs words. Display the loss and the training accuracy every 1000 iteration. Save the model at the end of training in the **lstm_model** folder

In [33]:
# Initializing the variables
start_time = time.time()
init = tf.global_variables_initializer()

# Create a summary to monitor cost and accuracy tensor
tf.summary.scalar("Loss", cost)
tf.summary.scalar("Accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()

# Initialize the saver
model_saver = tf.train.Saver()

print("Start Training")
##############################################

with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(epochs):
        avg_cost = 0.
        offset = epoch % (len(train_data)-n_input)
        # 3 words are taken from the training data, encoded to integer to form the input vector
        symbols_in_keys = np.array([ [dictionary[ str(train_data[i])]] for i in range(offset, offset+n_input) ])
        symbols_in_keys = symbols_in_keys.reshape(-1,n_input,1)

        # creation of the one-hot vector for training labels
        symbols_out_onehot = np.array(np.zeros([vocabulary_size], dtype=float))
        # putting to one the cell of the prediction
        symbols_out_onehot[dictionary[str(train_data[offset+n_input])]] = 1.0
        # reshaping 
        symbols_out_onehot = symbols_out_onehot.reshape(-1,vocabulary_size)

        # running the session
        _, acc, loss, summary, onehot_pred = sess.run([optimizer, accuracy, cost, merged_summary_op, pred], 
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        summary_writer.add_summary(summary, epoch)
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1))
            print("\t\t=====> Loss=", "{:.9f}".format(loss))
            print("\t\t=====> Accuracy=", "{:.9f}".format(acc))

    ##############################################
    print("End Of training Finished!")
    print("time: ",time.time() - start_time)
    print("For tensorboard visualisation run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("and point your web browser to the returned link")
    ##############################################
    model_saver.save(sess, 'lstm_model/my_model')
    ##############################################
    print("Model saved")


Start Training
Epoch:  1000
		=====> Loss= 2.814431667
		=====> Accuracy= 0.000000000
Epoch:  2000
		=====> Loss= 2.289624453
		=====> Accuracy= 1.000000000
Epoch:  3000
		=====> Loss= 2.784577847
		=====> Accuracy= 0.000000000
Epoch:  4000
		=====> Loss= 3.407334805
		=====> Accuracy= 0.000000000
Epoch:  5000
		=====> Loss= 1.719724536
		=====> Accuracy= 0.000000000
Epoch:  6000
		=====> Loss= 0.869452834
		=====> Accuracy= 1.000000000
Epoch:  7000
		=====> Loss= 1.474345684
		=====> Accuracy= 1.000000000
Epoch:  8000
		=====> Loss= 3.078511715
		=====> Accuracy= 0.000000000
Epoch:  9000
		=====> Loss= 0.179061979
		=====> Accuracy= 1.000000000
Epoch:  10000
		=====> Loss= 0.227435589
		=====> Accuracy= 1.000000000
Epoch:  11000
		=====> Loss= 0.266085327
		=====> Accuracy= 1.000000000
Epoch:  12000
		=====> Loss= 0.047968145
		=====> Accuracy= 1.000000000
Epoch:  13000
		=====> Loss= 0.097799979
		=====> Accuracy= 1.000000000
Epoch:  14000
		=====> Loss= 0.083410762
		=====> Accuracy

## Part 4 : Test your model 

### 3.1. Next word prediction

Load your model (using the model_saved variable given in the training session) and test the sentences :
- 'get a little' 
- 'nobody tried to'
- Try with other sentences using words from the stroy's vocabulary. 

In [35]:
sentences = ['a liar come','get a little','nobody tried to']

with tf.Session() as sess:
    # Initialize variables
    sess.run(init)
    model_saver = tf.train.Saver()
    model_saver.restore(sess, 'lstm_model/my_model')
    
    for sentence in sentences:
        try:
            words = sentence.split(' ')
            symbols_in_keys = [float(dictionary[str(words[i])]) for i in range(len(words))]    
            keys = np.reshape(np.array(symbols_in_keys), (-1, n_input, 1))
            onehot_pred = sess.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            print(sentence,'\n\n')
        except:
            print("Word not in dictionary")



INFO:tensorflow:Restoring parameters from lstm_model/my_model
a liar come the 


get a little will 


nobody tried to the 




### 3.2. More fun with the Fable Writer !

You will use the RNN/LSTM model learned in the previous question to create a
new story/fable.
For this you will choose 3 words from the dictionary which will start your
story and initialize your network. Using those 3 words the RNN will generate
the next word or the story. Using the last 3 words (the newly predicted one
and the last 2 from the input) you will use the network to predict the 5
word of the story.. and so on until your story is 5 sentence long. 
Make a point at the end of your story. 
To implement that, you will use the test function. 

In [37]:
sentence = 'the wise liar'
words = sentence.split(' ')
number_of_sentences = 5

with tf.Session() as sess:
    # Initialize variables
    sess.run(init)
    model_saver = tf.train.Saver()
    model_saver.restore(sess, 'lstm_model/my_model')
    s_count = 0
    print(sentence,end=' ')
    while(s_count<number_of_sentences):
        prediction = test(' '.join(words),sess)
        sentence = "%s %s" % (sentence,prediction)
        print(prediction,end=' ')
        words = words[1:]
        words.append(prediction)
        if prediction == '.':
            s_count += 1
#             print(s_count)




INFO:tensorflow:Restoring parameters from lstm_model/my_model
the wise liar of the village said : a liar will not be believed , and again the villagers , who had been fooled twice before , thought the boy was again deceiving them , and nobody stirred to come to his help . so the wolf made a good meal off the boy's flock , and when the boy complained , the wise man of the village said : a liar will not be believed , and again the villagers , who had been fooled twice before , thought the boy was again deceiving them , and nobody stirred to come to his help . so the wolf made a good meal off the boy's flock , and when the boy complained , the wise man of the village said : a liar will not be believed , and again the villagers , who had been fooled twice before , thought the boy was again deceiving them , and nobody stirred to come to his help . so the wolf made a good meal off the boy's flock , and when the boy complained , the wise man of the village said : a liar will not be believed ,

### 3.3. Play with number of inputs

The number of input in our example is 3, see what happens when you use other number (1 and 5)

Your answer goes here